In [3]:
import spacy
import numpy as np
import pandas as pd
import os
%matplotlib notebook

In [2]:
nlp = spacy.load('en')

In [37]:
doc = nlp("this is a text. and some   other!")
sentences = len(list(doc.sents))
words = sum([1 for token in doc if not token.is_space if not token.is_punct])
l
print(sentences)
print(words)
print(["+{}+".format(len(token)) for token in doc])
print(["+{}+".format(token) for token in doc])


2
7
['+4+', '+2+', '+1+', '+4+', '+1+', '+3+', '+4+', '+2+', '+5+', '+1+']
['+this+', '+is+', '+a+', '+text+', '+.+', '+and+', '+some+', '+  +', '+other+', '+!+']


In [11]:
# /Users/panayiotis/ETH/law/problem_sets/p1/data
# for every case .txt in that directory keep 'id', 'number of sentences' 'words', 'letters' and make a histogram
path_inputs = os.path.join('data', 'cases')
path_labels = os.path.join('data')

df = pd.DataFrame(columns=['ID', 'Sentences', 'Words', 'Letters'])

In [89]: s = df.iloc[3]

In [90]: df.append(s, ignore_index=True)

i = 0
for filename in os.listdir(path_inputs):
    i += 1
    if i > 3:
        break
    with open(os.path.join(path_inputs, filename)) as f:
        doc = nlp(f.read())
        
        

# df1[['year','citeCount']].hist()
# df1 = pd.read_csv('death-penalty-cases.csv')

 STONE , Circuit Judge.
This is an action to recover reparation for a freight overcharge.
The matter was first presented to the Interstate Commerce Commission, which found there had been an overcharge and declared reparation.
The railway having failed to pay the amount found due by the Commission, the present suit resulted.
From the judgment according reparation this appeal is brought.
The facts here are not in dispute and are as follows: These shipments were of carloads of grain originating on various points in Minnesota, North and South Dakota, upon the line of appellant.
All of them were consigned to Minneapolis and there later sent (by reconsignment without breakage of bulk in the same cars) to Superior, Wis. Appellant had in force rate tariffs prescribing certain rates from the points of origin of these shipments to Superior, rates to Minneapolis, and "proportional" rates from Minneapolis to Superior.
In the instance of each shipment there was a shorter mileage route from the poin